## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-16-15-52-27 +0000,nyt,Live Updates: Trump Backs Plan to Cede Land fo...,https://www.nytimes.com/live/2025/08/16/world/...
1,2025-08-16-15-50-56 +0000,nyt,Environmental Groups Face ‘Generational’ Setba...,https://www.nytimes.com/2025/08/16/climate/env...
2,2025-08-16-15-47-13 +0000,nypost,Knife-wielding thief on the loose after slashi...,https://nypost.com/2025/08/16/us-news/knife-wi...
3,2025-08-16-15-45-36 +0000,nyt,"Evo Morales, Barred from Bolivia’s Election, U...",https://www.nytimes.com/2025/08/15/world/ameri...
4,2025-08-16-15-45-23 +0000,nypost,NYPD detective cleared after rumor spread that...,https://nypost.com/2025/08/16/us-news/detectiv...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2337/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
2,trump,68
99,putin,42
8,ukraine,19
126,new,17
97,summit,16


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
57,2025-08-16-11-41-01 +0000,nyt,"After Trump-Putin Alaska Summit, Europeans Wor...",https://www.nytimes.com/2025/08/16/world/europ...,222
235,2025-08-15-15-56-48 +0000,nyt,"How Trump’s Meeting With Putin Could Unfold, a...",https://www.nytimes.com/2025/08/15/us/politics...,219
95,2025-08-16-08-01-18 +0000,nyt,Trump and Putin had an isolated conversation i...,https://www.nytimes.com/live/2025/08/15/world/...,184
125,2025-08-16-01-50-10 +0000,bbc,"No ceasefire, no deal: What summit means for T...",https://www.bbc.com/news/articles/clyvd3gkg1po...,175
139,2025-08-16-00-14-50 +0000,nypost,"Putin tries to woo Trump at Alaska meeting, cl...",https://nypost.com/2025/08/15/us-news/putin-tr...,173


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
57,222,2025-08-16-11-41-01 +0000,nyt,"After Trump-Putin Alaska Summit, Europeans Wor...",https://www.nytimes.com/2025/08/16/world/europ...
179,59,2025-08-15-20-58-21 +0000,nypost,Trump reacts to Hillary Clinton saying she’d n...,https://nypost.com/2025/08/15/us-news/trump-re...
205,46,2025-08-15-18-27-02 +0000,nypost,Feds accuse New Orleans Mayor LaToya Cantrell ...,https://nypost.com/2025/08/15/us-news/mayor-la...
6,42,2025-08-16-15-37-30 +0000,nypost,NYC pol rips Jewish activist over photo op wit...,https://nypost.com/2025/08/16/us-news/nyc-pol-...
99,41,2025-08-16-06-31-42 +0000,wapo,Air Canada cancels hundreds of flights as atte...,https://www.washingtonpost.com/world/2025/08/1...
127,34,2025-08-16-01-16-30 +0000,nyt,D.C. Police Chief Retains Control of City Poli...,https://www.nytimes.com/2025/08/15/us/politics...
147,31,2025-08-15-23-20-11 +0000,nypost,Trump claims ‘great progress’ in Putin meeting...,https://nypost.com/2025/08/15/us-news/trump-cl...
125,30,2025-08-16-01-50-10 +0000,bbc,"No ceasefire, no deal: What summit means for T...",https://www.bbc.com/news/articles/clyvd3gkg1po...
23,23,2025-08-16-14-45-34 +0000,nypost,Macron says pro-Zelensky ‘Coalition of the Wil...,https://nypost.com/2025/08/16/us-news/macron-s...
78,23,2025-08-16-10-00-00 +0000,latimes,California approves an unprecedented plan to p...,https://www.latimes.com/environment/story/2025...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
